In [56]:
'''
Belousov-Zhabotinsky reaction:
A non-equilibrium chemical oscillator w/ components that exhibit periodic changes in concetration
'''

# imports
import numpy as np
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display
import cv2 as cv
from PIL import Image

In [77]:
Images = {"PITT":"pittscript.jpg","TREE":"tree.png", "CAT":"cat.jpg"}

I = Image.open(Images["CAT"])
I = np.array(cv.resize(np.array(I),(600,450)))
#I = np.array(cv.threshold(I,127,255,cv.THRESH_BINARY))
I = I[:,:,1]/255


# Width, height of the image
row, col = I.shape
grid = []
alpha, beta, gamma = .7, 1, 1

In [78]:
def update_grid(p,grid):

    # reacalculate the chemical concentrations across the grid over 1 time step
    q = (p + 1) % 2
    #print(q)
    s = np.zeros((3, row, col))
    m = np.ones((3,3))/9 # 3x3 averaging kernel
    for k in range(3):
        s[k] = convolve2d(grid[p,k], m, mode='same', boundary='wrap')
    
    # reaction equations
    grid[q,0] = s[0] + s[0]*(alpha*s[1] - gamma*s[2])
    grid[q,1] = s[1] + s[1]*(beta*s[2] - alpha*s[0])
    grid[q,2] = s[2] + s[2]*(gamma*s[0] - beta*s[1])

    np.clip(grid[q], 0, 1, grid[q])
    return grid


In [83]:

grid = np.random.random(size=(2, 3, row, col))

grid[0][0][:,:] = grid[0][0][:,:]*I
grid[0][1][:,:] = grid[0][0][:,:]*I
grid[0][2][:,:] = grid[0][0][:,:]*I

grid[1][0][:,:] = grid[0][0][:,:]*I
grid[1][1][:,:] = grid[0][0][:,:]*I
grid[1][2][:,:] = grid[0][0][:,:]*I


In [84]:
nframes = 300

fig, ax = plt.subplots()
im = ax.imshow(grid[0, 0], cmap = 'jet')
ax.axis('off')

def animate(i, grid):
    grid = update_grid(i % 2, grid)
    im.set_array(grid[i % 2, 0])
    progress.progress += 1
    return im

progress = display.ProgressBar(nframes)
progress.display()

anim = FuncAnimation(fig, animate, frames=nframes, interval=30, blit=False, fargs=(grid,))
video = anim.to_html5_video()
plt.close()
display.HTML(video)

[============================================================] 301/300